## Nigerian Food Price Prediction - Notebook 2

## Objective
This notebook will:

- Load the raw dataset

- Create modular functions to process any state(not just 1 state like in notebook 1):

  - Aggregate market-level prices → state-level prices per day

  - Handle missing values

  - Create per-kg price columns for each commodity
 
- Demonstrate usage for Lagos (but works for any state)

- Compare with filtered dataset from notebook 1 


This makes the workflow scalable: you can add new states without rewriting code

## 2. Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("Libraries imported successfully!")


Libraries imported successfully!


## 3. Load data

In [2]:
# Load the dataset
df = pd.read_csv('../data_raw/NGA_market_2007 to 2025.csv', parse_dates=['price_date'])

print("Dataset loaded. Shape:", df.shape)
df.head()


Dataset loaded. Shape: (12597, 95)


,ISO3,country,adm1_name,adm2_name,mkt_name,lat,lon,geo_id,price_date,year,month,currency,components,start_dense_data,last_survey_point,data_coverage,data_coverage_recent,index_confidence_score,spatially_interpolated,bread,cassava_meal,cowpeas,gari,groundnuts,maize,millet,rice,sorghum,yam,o_bread,h_bread,l_bread,c_bread,inflation_bread,trust_bread,o_cassava_meal,h_cassava_meal,l_cassava_meal,c_cassava_meal,inflation_cassava_meal,trust_cassava_meal,o_cowpeas,h_cowpeas,l_cowpeas,c_cowpeas,inflation_cowpeas,trust_cowpeas,o_gari,h_gari,l_gari,c_gari,inflation_gari,trust_gari,o_groundnuts,h_groundnuts,l_groundnuts,c_groundnuts,inflation_groundnuts,trust_groundnuts,o_maize,h_maize,l_maize,c_maize,inflation_maize,trust_maize,o_millet,h_millet,l_millet,c_millet,inflation_millet,trust_millet,o_rice,h_rice,l_rice,c_rice,inflation_rice,trust_rice,o_sorghum,h_sorghum,l_sorghum,c_sorghum,inflation_sorghum,trust_sorghum,o_yam,h_yam,l_yam,c_yam,inflation_yam,trust_yam,o_food_price_index,h_food_price_index,l_food_price_index,c_food_price_index,inflation_food_price_index,trust_food_price_index
0,NGA,Nigeria,Abia,Oboma Ngwa,Aba,5.15,7.36,gid_5150000073600000,2007-01-01,2007,1,NGN,"bread (1 Unit, Index Weight = 1), cassava_meal...",Jan 2007,Jan 2023,12.01,0,0.96,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.09,39.31,36.86,38.09,NaN,7.2,10147.74,10302.97,9829.78,10165.96,NaN,9.2,10576.87,11018.41,10080.64,10574.63,NaN,6.6,7873.09,8059.34,7656.14,7912.79,NaN,9.3,25225.53,25539.31,24390.68,25152.93,NaN,8.8,3353.65,3404.27,3216.58,3363.81,NaN,9.3,4560.92,4666.26,4421.32,4587.98,NaN,8.9,6164.99,6235.31,6031.07,6209.82,NaN,9.5,4233.74,4356.45,4095.72,4266.28,NaN,8.3,61.01,64.19,59.13,61.74,NaN,9.2,0.47,0.48,0.46,0.48,NaN,9.4
1,NGA,Nigeria,Abia,Oboma Ngwa,Aba,5.15,7.36,gid_5150000073600000,2007-02-01,2007,2,NGN,"bread (1 Unit, Index Weight = 1), cassava_meal...",Jan 2007,Jan 2023,12.01,0,0.96,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.08,39.31,36.86,38.39,NaN,7.2,10084.45,10321.47,9627.00,9627.00,NaN,9.2,10547.29,11016.07,9214.69,9214.69,NaN,6.6,7897.36,8099.98,7562.43,7562.43,NaN,9.3,24893.14,25465.80,23673.78,23673.78,NaN,8.8,3320.46,3414.59,3037.84,3037.84,NaN,9.3,4570.74,4693.94,4182.67,4182.67,NaN,8.9,6177.79,6280.65,5918.40,5918.40,NaN,9.5,4258.57,4389.94,3748.06,3748.06,NaN,8.3,62.40,64.96,59.25,59.25,NaN,9.2,0.47,0.48,0.45,0.45,NaN,9.4
2,NGA,Nigeria,Abia,Oboma Ngwa,Aba,5.15,7.36,gid_5150000073600000,2007-03-01,2007,3,NGN,"bread (1 Unit, Index Weight = 1), cassava_meal...",Jan 2007,Jan 2023,12.01,0,0.96,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.49,39.63,37.35,37.90,NaN,7.2,9441.19,9704.99,9185.07,9704.99,NaN,9.2,8759.96,9214.80,8317.34,9214.80,NaN,6.6,7458.60,7652.24,7264.96,7614.74,NaN,9.3,23141.30,23840.14,22442.47,23805.37,NaN,8.8,2946.76,3043.77,2849.76,3041.09,NaN,9.3,4072.13,4191.93,3959.82,4191.93,NaN,8.9,5796.59,5929.94,5663.24,5923.32,NaN,9.5,3653.49,3788.04,3518.95,3752.11,NaN,8.3,59.06,61.52,56.61,59.43,NaN,9.2,0.44,0.45,0.43,0.45,NaN,9.4
3,NGA,Nigeria,Abia,Oboma Ngwa,Aba,5.15,7.36,gid_5150000073600000,2007-04-01,2007,4,NGN,"bread (1 Unit, Index Weight = 1), cassava_meal...",Jan 2007,Jan 2023,12.01,0,0.96,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.75,38.79,36.71,38.31,NaN,7.2,9690.08,9938.50,9441.67,9711.28,NaN,9.2,9000.20,9423.97,8576.42,9148.48,NaN,6.6,7617.10,7822.72,7411.47,7624.58,NaN,9.3,23684.74,24281.06,23088.42,23635.41,NaN,8.8,3012.11,3107.03,2917.19,3018.89,NaN,9.3,4140.71,4299.48,3981.94,4154.57,NaN,8.9,5843.83,5963.77,5723.89,5894.22,NaN,9.5,3667.32,3813.36,3521.28,3727.59,NaN,8.3,60.13,61.91,58.35,59.05,NaN,9.2,0.45,0.46,0.44,0.45,NaN,9.4
4,NGA,Nigeria,Abia,Oboma Ngwa,Aba,5.15,7.36,gid_5150000073600000,2007-05-01,2007,5,NGN,"bread (1 Unit, Index Weight = 1), cassava_meal...",Jan 2007,Jan 2023,12.01,0,0.96,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.48,39.46,37.50,37.82,NaN,7.2,9746.52,9950.41,9542.62,9611.01,NaN,9.2,9085.91,9508.10,8663.71,9104.57,NaN,6.6,7661.86,7833.59,7490.13,7549.90,NaN,9.3,23584.98,24067.33,23

## 4. Define functions

## 4.1 Aggregate markets into a state-level dataset, filter to selected years

In [3]:
def process_state(df, state_name, start_year=2020, end_year=2025):
    """
    Filter raw food price data for a specific state and year range, 
    select relevant columns, aggregate multiple markets into one state-level price per commodity, 
    handle missing values, and return clean dataframe.
    """
    # Step 1: Filter by state
    df_state = df[df['adm1_name'] == state_name].copy()
    
    # Step 2: Filter by year range
    df_state = df_state[(df_state['year'] >= start_year) & (df_state['year'] <= end_year)]
    
    # Step 3: Keep relevant columns
    closing_cols = [
        'c_bread', 'c_cassava_meal', 'c_cowpeas', 'c_gari', 'c_groundnuts',
        'c_maize', 'c_millet', 'c_rice', 'c_sorghum', 'c_yam', 'c_food_price_index'
    ]
    df_state = df_state[['adm1_name', 'price_date'] + closing_cols]
    
    # Step 4: Aggregate multiple markets per day (mean)
    df_state = df_state.groupby(['adm1_name', 'price_date'])[closing_cols].mean().reset_index()
    
    # Step 5: Handle missing values (forward-fill then backward-fill per state)
    df_state = df_state.groupby('adm1_name').ffill().bfill()
    
    return df_state


## 4.2 Create per-kg price columns

In [4]:
def create_per_kg_columns(df_state, per_kg_mapping):
    """
    Convert commodity prices to per-kg prices using the mapping provided.
    
    Parameters:
        df_state (pd.DataFrame): Aggregated state-level dataset
        per_kg_mapping (dict): {commodity_column: kg_multiplier}
        
    Returns:
        df_state (pd.DataFrame): Dataset with additional per-kg columns
    """
    for col, multiplier in per_kg_mapping.items():
        df_state[f'{col}_per_kg'] = df_state[col] * multiplier
    return df_state

In [5]:
# Per-kg multipliers based on dataset's 'components' column
per_kg_mapping = {
    'c_bread': 1,             # bread per unit, keep as is
    'c_cassava_meal': 0.01,   # 100kg → per kg
    'c_cowpeas': 0.01,        # 100kg → per kg
    'c_gari': 0.01,           # 100kg → per kg
    'c_groundnuts': 0.01,     # 100kg → per kg
    'c_maize': 0.01,          # 100kg → per kg
    'c_millet': 0.01,         # 100kg → per kg
    'c_rice': 0.02,           # 50kg → per kg
    'c_sorghum': 0.01,        # 100kg → per kg
    'c_yam': 1,               # 1kg → per kg
}


## 5. Run modular pipeline on Lagos

In [7]:
# Aggregate Lagos
df_lagos_state = process_state(df, "Lagos")

# Create per-kg columns
df_lagos_state = create_per_kg_columns(df_lagos_state, per_kg_mapping)

# Check dataset
print(df_lagos_state.shape)
df_lagos_state.head()


(65, 22)


,price_date,c_bread,c_cassava_meal,c_cowpeas,c_gari,c_groundnuts,c_maize,c_millet,c_rice,c_sorghum,c_yam,c_food_price_index,c_bread_per_kg,c_cassava_meal_per_kg,c_cowpeas_per_kg,c_gari_per_kg,c_groundnuts_per_kg,c_maize_per_kg,c_millet_per_kg,c_rice_per_kg,c_sorghum_per_kg,c_yam_per_kg
0,2020-01-01,609.76,8700.00,16450.00,5034.31,26650.00,10663.99,12800.00,22475.00,11577.97,189.79,1.17,609.76,87.0000,164.5000,50.3431,266.5000,106.6399,128.0000,449.5000,115.7797,189.79
1,2020-02-01,609.76,8700.00,16300.00,5756.61,26900.00,10650.00,13225.00,22300.00,11475.00,196.25,1.18,609.76,87.0000,163.0000,57.5661,269.0000,106.5000,132.2500,446.0000,114.7500,196.25
2,2020-03-01,609.76,9964.15,17287.50,7028.40,28475.00,11912.50,14012.50,26525.00,13225.00,211.38,1.28,609.76,99.6415,172.8750,70.2840,284.7500,119.1250,140.1250,530.5000,132.2500,211.38
3,2020-04-01,664.74,11763.00,18466.67,9691.85,31786.67,13210.00,15153.33,28348.97,14136.67,234.98,1.41,664.74,117.6300,184.6667,96.9185,317.8667,132.1000,151.5333,566.9794,141.3667,234.98
4,2020-05-01,605.79,13769.13,18750.00,12397.37,33722.58,14022.58,17290.32,28093.55,14674.19,282.73,1.45,605.79,137.6913,187.5000,123.9737,337.2258,140.2258,172.9032,561.8710,146.7419,282.73


## 6. Notebook comparison

In [12]:
# Load Notebook 1 cleaned dataset
df_lagos_notebook1 = pd.read_csv('../data_processed/lagos_filtered.csv')

In [13]:
# Compare shapes
print("Notebook 1 shape:", df_lagos_notebook1.shape)
print("Modular function shape:", df_lagos_state.shape)

Notebook 1 shape: (65, 22)
Modular function shape: (65, 22)


In [14]:
# Compare first few rows
print("\nNotebook 1 sample:")
display(df_lagos_notebook1.head())

print("\nModular function sample:")
display(df_lagos_state.head())


Notebook 1 sample:


,price_date,c_bread,c_cassava_meal,c_cowpeas,c_gari,c_groundnuts,c_maize,c_millet,c_rice,c_sorghum,c_yam,c_food_price_index,c_bread_per_kg,c_cassava_meal_per_kg,c_cowpeas_per_kg,c_gari_per_kg,c_groundnuts_per_kg,c_maize_per_kg,c_millet_per_kg,c_rice_per_kg,c_sorghum_per_kg,c_yam_per_kg
0,2020-01-01,609.76,8700.00,16450.00,5034.31,26650.00,10663.99,12800.00,22475.00,11577.97,189.79,1.17,609.76,87.0000,164.5000,50.3431,266.5000,106.6399,128.0000,449.5000,115.7797,189.79
1,2020-02-01,609.76,8700.00,16300.00,5756.61,26900.00,10650.00,13225.00,22300.00,11475.00,196.25,1.18,609.76,87.0000,163.0000,57.5661,269.0000,106.5000,132.2500,446.0000,114.7500,196.25
2,2020-03-01,609.76,9964.15,17287.50,7028.40,28475.00,11912.50,14012.50,26525.00,13225.00,211.38,1.28,609.76,99.6415,172.8750,70.2840,284.7500,119.1250,140.1250,530.5000,132.2500,211.38
3,2020-04-01,664.74,11763.00,18466.67,9691.85,31786.67,13210.00,15153.33,28348.97,14136.67,234.98,1.41,664.74,117.6300,184.6667,96.9185,317.8667,132.1000,151.5333,566.9794,141.3667,234.98
4,2020-05-01,605.79,13769.13,18750.00,12397.37,33722.58,14022.58,17290.32,28093.55,14674.19,282.73,1.45,605.79,137.6913,187.5000,123.9737,337.2258,140.2258,172.9032,561.8710,146.7419,282.73



Modular function sample:


,price_date,c_bread,c_cassava_meal,c_cowpeas,c_gari,c_groundnuts,c_maize,c_millet,c_rice,c_sorghum,c_yam,c_food_price_index,c_bread_per_kg,c_cassava_meal_per_kg,c_cowpeas_per_kg,c_gari_per_kg,c_groundnuts_per_kg,c_maize_per_kg,c_millet_per_kg,c_rice_per_kg,c_sorghum_per_kg,c_yam_per_kg
0,2020-01-01,609.76,8700.00,16450.00,5034.31,26650.00,10663.99,12800.00,22475.00,11577.97,189.79,1.17,609.76,87.0000,164.5000,50.3431,266.5000,106.6399,128.0000,449.5000,115.7797,189.79
1,2020-02-01,609.76,8700.00,16300.00,5756.61,26900.00,10650.00,13225.00,22300.00,11475.00,196.25,1.18,609.76,87.0000,163.0000,57.5661,269.0000,106.5000,132.2500,446.0000,114.7500,196.25
2,2020-03-01,609.76,9964.15,17287.50,7028.40,28475.00,11912.50,14012.50,26525.00,13225.00,211.38,1.28,609.76,99.6415,172.8750,70.2840,284.7500,119.1250,140.1250,530.5000,132.2500,211.38
3,2020-04-01,664.74,11763.00,18466.67,9691.85,31786.67,13210.00,15153.33,28348.97,14136.67,234.98,1.41,664.74,117.6300,184.6667,96.9185,317.8667,132.1000,151.5333,566.9794,141.3667,234.98
4,2020-05-01,605.79,13769.13,18750.00,12397.37,33722.58,14022.58,17290.32,28093.55,14674.19,282.73,1.45,605.79,137.6913,187.5000,123.9737,337.2258,140.2258,172.9032,561.8710,146.7419,282.73


## 7. Save state-level dataset

In [15]:
output_path = "../data_processed/lagos_modular.csv"
df_lagos_state.to_csv(output_path, index=False)
print(f"Lagos state-level modular dataset saved at: {output_path}")


Lagos state-level modular dataset saved at: ../data_processed/lagos_modular.csv
